In [ ]:
from tune_util import *
import numpy as np
import pickle
import functools
import cv2

import tempfile


In [ ]:
import random
import configparser
import os
episodes_config = configparser.ConfigParser(
    allow_no_value=True, inline_comment_prefixes=";"
)
episodes_config.read(os.path.join(os.getcwd(), "params/episode_params_val.ini"))
series = episodes_config.sections()[1:]

In [ ]:
def_num = np.array([10,1,3,1000,4,1,0.8,2,0.35,2,3])
def_num_log = np.array(list(np.log(def_num)) + [0.0])

bounds = np.array([[0.001,1000],[0.001,1000],[0.001,1000],[0.001,1000],
          [2,25],[0.001,100],[0.1,10],[0.1,10],[0.1,10],[1,10],[1,10],[-0.2,0.2]])
bounds = list(np.log(bounds[:-1])) + [bounds[-1]]

In [ ]:
my_log = []
res = exec_seqs(def_num_log,'test_eval',['t_ETH1','--render'],log_file=my_log)
res

In [ ]:
cv2.destroyAllWindows()

REND_FPS = 10
rend_time_delay = int(round(1000/REND_FPS))

pair_comps = []

import cma
es = cma.CMAEvolutionStrategy(def_num_log,0.1)
inits = es.ask()
chosen = [random.choice(series),'--render']
for sol_i,init in enumerate(inits):
    res = exec_seqs(init,'mandorsa_{:03d}'.format(sol_i),chosen,log_file=my_log)
ind_r = []
for sol_i,i in enumerate(sorted(glob.glob(os.path.join(os.getcwd(),'tests/socnav/mandorsa_*')))):
    params = inits[sol_i]

    pkl = glob.glob(os.path.join(i,'*/*.pkl'))[0]
    with open(pkl,'rb') as fp:
        res = pickle.load(fp)

    cost, costs_vec = eval_heuristic(res)

    jpgs = sorted(glob.glob(pkl.replace('episode_score_','').replace('.pkl','_obs_obs*.jpg')))
    read_data = [cv2.imread(_) for _ in jpgs]
    ind_r.append((cost,params,res,read_data))

def compare(i,j):
    cv2.destroyAllWindows()

    c1,params1,_,set1 = ind_r[i]
    c2,params2,_,set2 = ind_r[j]

    s12_s = list(set1[0].shape)
    s12_s[1]*=2
    while True:
        cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                   np.stack([set1[len(set1)-1],set2[len(set2)-1]],axis=1).reshape(s12_s))
        a = cv2.waitKey(rend_time_delay)

        if a == ord('1'):
            pair_comps.append([1,params1,params2])
            return 1
        if a == ord('2'):
            pair_comps.append([2,params2,params1])
            return -1
        if a == ord('3'):
            pair_comps.append([3] + ([params1,params2] if (c1 < c2) else [params2,params1]))
            return 2*(c1 <c2)-1
        if a == ord('q'):
            return
        if a == ord('g'):
            break

    while True:
        for k in range(max(len(set1),len(set2))):
            cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                       np.stack([set1[min(k,len(set1)-1)],set2[min(k,len(set2)-1)]],axis=1).reshape(s12_s))
            a = cv2.waitKey(rend_time_delay)

            if a == ord('1'):
                pair_comps.append([1,params1,params2])
                return 1
            if a == ord('2'):
                pair_comps.append([2,params2,params1])
                return -1
            if a == ord('3'):
                pair_comps.append([3] + ([params1,params2] if (c1 < c2) else [params2,params1]))
                return 2*(c1 <c2)-1
            if a == ord('q'):
                return
            else:
                continue
res = sorted(list(range(len(ind_r))),key=functools.cmp_to_key(compare))
cv2.destroyAllWindows()


In [ ]:
pair_comps

In [ ]:
from GP import GaussianProcess
import numpy as np
import scipy.optimize as opt
cv2.destroyAllWindows()

def findBestQuery(gp):
    def negative_info_gain(x):
        return -1*gp.objectiveEntropy(x)
    x0 = np.array(list(gp.initialPoint)*2) * (np.random.rand(gp.dim*2)*0.2 + 0.9)
    # Let's now find the optimal query within the bounds (-2,2) for each dimension
    opt_res = opt.fmin_l_bfgs_b(negative_info_gain, x0=x0, bounds=np.vstack([np.array(bounds),np.array(bounds)]),
                                approx_grad=True, iprint=-1)
    return opt_res[0], -opt_res[1]

initialPoint =  def_num_log #np.array(inits).mean(0)  # where we assume the function value is 0
theta = 1. # hyperparameter
noise_level = 0.1 # noise parameter that corresponds to \sqrt{2}\sigma in the paper

gp = GaussianProcess(initialPoint, theta, noise_level)
for _,p1,p2 in pair_comps:
    gp.updateParameters([p1,p2], 1)
while True:
    next_x0,cost = findBestQuery(gp)
    cfg1 = np.array(next_x0[:gp.dim])
    cfg2 = np.array(next_x0[gp.dim:])
    chosen = [random.choice(series),'--render']
    res1 = exec_seqs(cfg1,'dorsa_pair1',chosen,log_file=my_log)
    res2 = exec_seqs(cfg2,'dorsa_pair2',chosen,log_file=my_log)
    ind_r = []
    for sol_i,i in enumerate(sorted(glob.glob(os.path.join(os.getcwd(),'tests/socnav/dorsa_pair*')))):
        pkl = glob.glob(os.path.join(i,'*/*.pkl'))[0]
        with open(pkl,'rb') as fp:
            res = pickle.load(fp)

        cost, costs_vec = eval_heuristic(res)

        jpgs = sorted(glob.glob(pkl.replace('episode_score_','').replace('.pkl','_obs_obs*.jpg')))
        read_data = [cv2.imread(_) for _ in jpgs]
        ind_r.append((cost,params,res,read_data))

    c1,params1,_,set1 = ind_r[0]
    c2,params2,_,set2 = ind_r[1]

    s12_s = list(set1[0].shape)
    s12_s[1]*=2
    res_v = 0
    while True:
        cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                   np.stack([set1[len(set1)-1],set2[len(set2)-1]],axis=1).reshape(s12_s))
        a = cv2.waitKey(rend_time_delay)

        if a == ord('1'):
            pair_comps.append([1,params1,params2])
            res_v = 1
            break
        if a == ord('2'):
            pair_comps.append([2,params2,params1])
            res_v = -1
            break
        if a == ord('3'):
            pair_comps.append([3] + ([params1,params2] if (c1 < c2) else [params2,params1]))
            resv = 1 if c1 < c2 else -1
            break
        if a == ord('q'):
            raise
        if a == ord('g'):
            break
    if res_v == 0:
        run_video = True
        while run_video:
            for k in range(max(len(set1),len(set2))):
                cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                           np.stack([set1[min(k,len(set1)-1)],set2[min(k,len(set2)-1)]],axis=1).reshape(s12_s))
                a = cv2.waitKey(rend_time_delay)

                if a == ord('1'):
                    pair_comps.append([1,params1,params2])
                    res_v = 1
                    run_video = False
                    break
                if a == ord('2'):
                    pair_comps.append([2,params2,params1])
                    res_v = -1
                    run_video = False
                    break
                if a == ord('3'):
                    pair_comps.append([3] + ([params1,params2] if (c1 < c2) else [params2,params1]))
                    resv = 1 if c1 < c2 else -1
                    run_video = False
                    break
                if a == ord('q'):
                    raise
    gp.updateParameters([params1,params2], res_v)
    cv2.destroyAllWindows()


In [ ]:
res_lookup = {}
for c in my_log:
    res_lookup[tuple(c['config_used'])] = eval_heuristic(c)[1]

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
reg_X = []
reg_y = []
for i,data in enumerate(pair_comps):
    _,p1,p2 = data
    if (i%2) == 0:
        reg_X.append(np.array(res_lookup[tuple(p1)])-np.array(res_lookup[tuple(p2)]))
        reg_y.append(1)
    else:
        reg_X.append(np.array(res_lookup[tuple(p2)])-np.array(res_lookup[tuple(p1)]))
        reg_y.append(0)
clf = LogisticRegressionCV(Cs=3,fit_intercept=False)
clf.fit(reg_X,reg_y)
cfgs = [_ for _ in res_lookup.keys()]
rewards = [res_lookup[_] for _ in cfgs]
pred_v = (np.array(rewards) * clf.coef_).sum(1)
np.argmax(pred_v),pred_v

In [ ]:
results = {'cfg':np.array(cfgs[np.argmax(pred_v)]),'log':my_log,'comps':pair_comps,'coeff':clf.coef_}
import glob
import pickle
N=len(glob.glob('mandorsa_results*.pkl'))
with open('manorsa_results_{}.pkl'.format(N),'wb') as fp:
    pickle.dump(results,fp)